In [1]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from pgmpy.estimators import HillClimbSearch
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BicScore
from pgmpy.base import DAG
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from pgmpy.base import DAG
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
from rpy2.robjects import globalenv
from rpy2.robjects.packages import importr
from pgmpy.estimators import K2Score
from rpy2.robjects import conversion, default_converter, pandas2ri
from rpy2.robjects.conversion import localconverter
from pgmpy.models import BayesianNetwork
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import rpy2.robjects as ro
from pgmpy.estimators import ExpectationMaximization as EM
from pgmpy.estimators import BayesianEstimator as BE
pandas2ri.activate()
from sklearn.metrics import mutual_info_score

In [2]:
def to_matrix(edges_dag,columns):
    #将图转换为矩阵
    edges_matrix = pd.DataFrame(np.zeros((len(columns),len(columns))),index = columns,columns=columns).astype(int)
    for edges in edges_dag:
        edges_matrix.loc[edges[0],edges[1]]+=1
    return edges_matrix
def to_dag(edges_matrix):
    edges_list = []
    for index in edges_matrix.index:
        for col in edges_matrix.columns:
            if edges_matrix.loc[index,col]!=0:
                edges_list.append((index,col))
            
    #将矩阵转换为图
    return edges_list
# Funtion to evaluate the learned model structures.
def get_f1_score(estimated_model, true_model):
    nodes = estimated_model.nodes()
    est_adj = nx.to_numpy_matrix(
        estimated_model.to_undirected(), nodelist=nodes, weight=None
    )
    true_adj = nx.to_numpy_matrix(
        true_model.to_undirected(), nodelist=nodes, weight=None
    )

    f1 = f1_score(np.ravel(true_adj), np.ravel(est_adj))
    print("F1-score for the model skeleton: ", f1)    
def print_dag(dag):
    if dag.has_node("label"):
        for edges in dag.get_ancestral_graph("label").edges:
            print(edges[0],"->",edges[1],";")
    else:
        for edges in dag.edges:
            print(edges[0],"->",edges[1],";")
def model_auc(prob,y_true):
    y_prob = []
    num = 0
    for i in range(len(prob)):
        y_prob.append(prob.iloc[i,y_true[i]])
    if y_predic[i]==y_true[i]:
        num+=1
    fpr, tpr, thresholds = roc_curve(y_true,y_predic)
    
    return auc(fpr, tpr)
def to_strength_list(edges_strength_martix):
    str_list = []
    for col in edges_strength_martix.columns:
        for idx in edges_strength_matrix.columns:
            if edges_strength_matrix[col][idx] < 0:
                str_list.append([idx,col,edges_strength_matrix[col][idx]])
    return str_list
def print_edges(ed):
    for i in ed:
        print(i[0],"->",i[1],";")

In [3]:
class client():
    """
    只改动模型训练部分
    black和whitelist 的名单修改
    
    """
    def __init__(self,data,cid,strength):
        #black 用R语言的存储方式存储·
        self.data = data
        self.cid = cid
        #初始化创建一个空的图
        self.model = DAG()
        self.edges_strength =  strength
        
    def get_model(self):
        #获得本地客户端网络结构
        return self.model
    def fit(self):
        #记录最好的模型
        # 代改动部分        
        
        self_r_data = ro.conversion.py2rpy(self.data)
        globalenv['self_r_data'] = self_r_data
        r_script="""
        library(bnlearn)
        
        
        datacols = names(self_r_data)
        for (i in 1:ncol(self_r_data)) { 
        if(i>45){
            self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]],levels = c(0,1))
        }else{
            self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]])
              }
        }
        dag = hc(self_r_data,score = sc,blacklist = bl_r)
        arcs = arc.strength(dag,self_r_data)
        
        """
        edges = r(r_script)
        self.arc_strength_list = pd.DataFrame(edges)
        self.model_edges = edges
        #创建新的模型，并添加结果
        self.model = DAG()
        for i in range(len(edges["from"])):
            #汇集频数
            self.model.add_edge(edges["from"][i],edges["to"][i])
          
            
#             #汇集强度
            self.edges_strength.loc[[edges["from"][i]],[edges["to"][i]]]=edges["strength"][i]
        
        return 1
    def set_parment(self):
        #从服务器中心get到网络结构
        #初始化model
        model.edges
    
        return 
    def update_model(self,fuse_dag):
        #代改动部分，每次也是创建一个新的贝叶斯网络实例进行白名单结构学习，并返回边模型，最好还是保持pgmpy的形式
        
        
        self.model = DAG()
        #设置白名单
        white_list = fuse_dag.edges()
        wl = []
        for edge in white_list:
            wl.append([edge[0],edge[1]])
        
        wl = pd.DataFrame(wl,columns=["from","to"])
        wl_r =ro.conversion.py2rpy(wl)
        globalenv['wl_r'] = wl_r
        self_r_data = ro.conversion.py2rpy(self.data)
        globalenv['self_r_data'] = self.data
        
        r_script="""
        library(bnlearn)
       
        datacols = names(self_r_data)
        for (i in 1:ncol(self_r_data)) { 
        if(i>45){
            self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]],levels = c(0,1))
        }else{
            self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]])
              }
        }
        dag = hc(self_r_data,score = sc,blacklist = bl_r,whitelist = wl_r)
        arcs = arc.strength(dag,self_r_data)
        
        """
        edges = r(r_script)
        self.model_edges = edges
      
        for i in range(len(edges["from"])):
            self.model.add_edge(edges["from"][i],edges["to"][i])
        

        
class sever():
    def __init__(self,fuse_num):
        self.fuse_num = fuse_num
    def Fd_caculate(self,edges_strength_matrix):
        #聚合方法
        #聚合的方式改为计算强度的阈值，并返回一个有无环图
        fuseDAG = DAG()
        #变为list，并且设置为df格式
        arc_ls = pd.DataFrame(to_strength_list(edges_strength_matrix.astype(np.float16)),columns=["from","to","strength"])
        #from和to的df格式并按大小排列
        arc_ls = arc_ls.sort_values(by="strength",ascending=False)
        #1.按照10个选择
        for  i in range(20):
            fuseDAG.add_edge(arc_ls.loc[i,"from"],arc_ls.loc[i,"to"])
            
   
        return  fuseDAG

In [4]:
DB_1 = pd.read_csv("c:data/site_16topaucoutlinePmmImpData_dis.csv").drop(columns="Unnamed: 0",axis = 1)
DB_1["label"].value_counts()
DB_1 = DB_1.astype(np.int16)

# 数据分割
hos_id = [420,142,122,435,390,227,195,243,403,141]
client_data = []
for hosid in hos_id:
    client_data.append(DB_1[DB_1["hospitalid"]==hosid ].iloc[:,:].drop(columns = ["hospitalid"],axis = 1))
    
feature = client_data[0].columns

#设置黑名单
bl = []
for i in feature:
    if i =="label":
        pass
    bl.append(["label",i])
for i in feature:
    if i =="age":
        pass
    bl.append([i,"age"])
for i in feature:
    if i =="sex":
        pass
    bl.append([i,"sex"])
for i in feature:
    if i =="race":
        pass
    bl.append([i,"race"])

bl = pd.DataFrame(bl,columns=["from","to"])
bl_r = ro.conversion.py2rpy(bl)
globalenv['bl_r'] = bl_r

In [5]:
#计算样本量比重
D = 19388
D_W = []
for i in range(10):
    D_W.append(len(client_data[i])/D)
D_W = np.float16(D_W)

In [6]:
def cross_validation(ed,DB,cid):
    
    #cid客户端号，寻找相应的训练数据
    auc_score = np.zeros((10,1))
    recall = np.zeros((10,1))
    acc = np.zeros((10,1))
# #    实例化对象
#     model= BayesianNetwork()
#     #创建边
    
#     model.add_edges_from(ed)
#     #确保数据的每一个都是新的
#     f_1 = model.nodes
        
#     f_2 = DB[cid].columns
#     data =DB[cid].copy(deep=True)
#     drop_name = []
#     for j in f_2:
#         if not j in f_1:
#             drop_name.append(j)
#     data = data.drop(columns = drop_name,axis = 1)
    
#     #分出训练集
#     train_data,test_data = train_test_split(DB[cid],test_size=0.3,random_state=2)
#     #训练
#     model.fit(train_data,estimator=BE)
    
    for i in range(len(DB)): 
        
        model_i= BayesianNetwork(ed.edges)
        #创建边
        data = DB[i]
        
        f_1 = model_i.nodes
        
        f_2 = data.columns
        #所有mode有的节点
        drop_name = []
        for j in f_2:
            if not j in f_1:
                drop_name.append(j)        
        data = data.drop(columns = drop_name,axis = 1)
        
        
        #创建离散变量说明
        this_dict = []

        for key,value in zip(state_names.keys(),state_names.values()):
            
            if key in f_1:

                this_dict.append((key,value))
        this_dict = dict(this_dict)
         #训练模型
        
        train_data,test_data = train_test_split(data,test_size=0.3,random_state=2)
        #得到y-true
        y_true = test_data.iloc[:,0]
        
        
        model_i.fit(train_data,estimator=BE,complete_samples_only=True,state_names=this_dict)
        
        #丢弃label获得
        
        test_data.drop(columns="label",axis =1,inplace=True)
      
        #将预测的值从df转换为list形式
        
        #在predict的时候不知道什么会影响结果
        y_pred = list(model_i.predict(test_data).iloc[:,0])
       
       #模型预测
        acc[i] = accuracy_score(y_true,y_pred)
        recall[i] = recall_score(y_true, y_pred)
        auc_score[i] = roc_auc_score(y_true,y_pred)
    print(acc)
    print(recall)
    print(auc_score)
    return acc,recall,auc_score

In [7]:
#将所有的离散状态记录下来，以字典的方式
state_names  ={
    "lable":[0,1],
    "age":[1,2,3,4,5,6,7,8],
    "sex":[1,2],
    "race":[1,2,3,4,5],
    "bmi":[1,2,3,4],
    "temperature":[1,2,3],
    "heartrate":[1,2,3],
    "respiration":[1,2,3],
    "SBP":[1,2,3,4,5],
    "DBP":[1,2,3,4,5],
    "paSystolic":[1,2,3],
    "paDiastolic":[1,2,3],
    "paMean":[1,2,3],
}
LAB_list = []
for i in range(1,61):
    LAB_list.append(("LAB"+str(i),[1,2,3]))
COM_list = []
for i in range(1,12):
    COM_list.append(("COM"+str(i),[0,1]))
PRO_list = []
for i in range(1,10):
    COM_list.append(("PRO"+str(i),[0,1]))
MED_list = []
for i in range(1,55):
    MED_list.append(("MED"+str(i),[0,1]))
state_names.update(LAB_list)
state_names.update(COM_list)
state_names.update(PRO_list)
state_names.update(MED_list)     

In [8]:
globalenv['sc'] = "k2"
sth = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
pandas2ri.activate()
# 结构训练
train_round = 10
round_num = 0
client_num = 10
clients = []
for i in range(10):
# 创建客户端，并且传入数据和结构学习模型
    clients.append(client(client_data[i],i,sth))
# 初始化服务器
sever_1 = sever(6)
global_edges_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
edges_strength_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
while(round_num<train_round):
    #利用二维矩阵传递边结构信息
    auc_list = []
    recall_list = []
    acc_list = []
    #开始训练
    #重置矩阵,设置评分
    global_edges_matrix.iloc[:,:] = 0
    edges_strength_matrix.iloc[:,:] = 0
    acc = 0
    recall = 0
    acu_score = 0
    #第一轮，先本地客户端建立初始模型
    if round_num == 0:
        
        print("初始化训练")
        for i in range(client_num):
            clients[i].fit()

            
            #计算初始性能，计算轮数的性能
# #             print("各个客户端的初始性能")
#             acc,recall,auc_score = cross_validation(clients[i].model,client_data,clients[i].cid)
#             auc_list.append(acu_score)
#             recall_list.append(recall)
#             acc_list.append(acc)

        print("初始auc",auc_list)
        print("初始recall",recall_list)
        print("初始acc",acc_list)
        auc_list = []
        recall_list = []
        acc_list = [] 
        
    #get到每个客户端边强度的总和
    arc_strength_list = []
    arc_weight_list = []
    #开始聚集客户端边的信息
    for i in range(client_num):
        #将客户端最好的模型边信息转换为矩阵形式
        
        global_edges_matrix+=to_matrix(clients[i].get_model().edges(),feature)
        #将矩阵转换为列表形式，并strength按列合并数据
        
        arc_strength_list.append(np.float16(pd.DataFrame(clients[i].arc_strength_list)["strength"].sum()))
        
#         edges_strength_matrix+=clients[i].edges_strength
    #计算每个图像的权
    sum_value = sum(arc_strength_list)
    
    for  i in range(client_num):
        arc_weight_list.append(arc_strength_list[i]/sum_value)
    #汇集带权重的边的信息
    for i in range(client_num):
        edges_strength_matrix+=clients[i].edges_strength*arc_weight_list[i]*D_W[i]
    
    #服务器开始汇集信息，并且将矩阵转换为Dag
#     print("未过滤的融合矩阵")
#     for index in edges_matrix.index:
#         for col in edges_matrix.columns:
#             if edges_matrix.loc[index,col]!=0:
#                 print(index,"->",col,"[label=",edges_matrix.loc[index,col],"];")
    
    fuseDAG = sever_1.Fd_caculate(edges_strength_matrix)
    print("第",round_num+1,"次迭代")
    #输出融合模型
#     print_dag(fuseDAG)
    #更新客户端模型
    print("更 新客户端")
    #更新客户端，就是将融合的DAG()，传给各个模型，并用bl从新训练
    
    for i in range(client_num):
        clients[i]. update_model(fuseDAG)
        
    round_num+=1
    


    
    
    



初始化训练
初始auc []
初始recall []
初始acc []
第 1 次迭代
更 新客户端
第 2 次迭代
更 新客户端
第 3 次迭代
更 新客户端
第 4 次迭代
更 新客户端
第 5 次迭代
更 新客户端
第 6 次迭代
更 新客户端
第 7 次迭代
更 新客户端
第 8 次迭代
更 新客户端
第 9 次迭代
更 新客户端
第 10 次迭代
更 新客户端


In [9]:
print_dag(clients[8].model)

LAB19 -> label ;
LAB19 -> COM5 ;
LAB19 -> COM7 ;
LAB19 -> LAB34 ;
COM1 -> label ;
COM1 -> COM11 ;
COM1 -> COM7 ;
COM1 -> PRO2 ;
COM2 -> label ;
COM2 -> COM8 ;
COM2 -> LAB25 ;
COM2 -> LAB3 ;
COM3 -> label ;
COM4 -> label ;
COM4 -> COM10 ;
COM4 -> LAB23 ;
COM5 -> label ;
COM5 -> COM11 ;
COM5 -> COM8 ;
COM5 -> PRO7 ;
COM5 -> COM7 ;
COM6 -> label ;
COM6 -> PRO3 ;
COM6 -> COM10 ;
COM6 -> LAB36 ;
COM6 -> LAB23 ;
COM7 -> label ;
COM7 -> COM8 ;
COM7 -> COM10 ;
COM7 -> COM11 ;
COM7 -> PRO3 ;
COM8 -> label ;
COM9 -> label ;
COM9 -> LAB23 ;
COM9 -> LAB14 ;
COM9 -> LAB35 ;
COM9 -> LAB56 ;
COM9 -> COM6 ;
COM10 -> label ;
COM10 -> COM3 ;
COM10 -> PRO8 ;
COM11 -> label ;
COM11 -> COM10 ;
COM11 -> PRO8 ;
PRO1 -> label ;
PRO1 -> LAB3 ;
PRO1 -> PRO3 ;
PRO1 -> COM1 ;
PRO1 -> COM8 ;
PRO1 -> COM7 ;
PRO1 -> LAB10 ;
PRO1 -> LAB19 ;
PRO2 -> label ;
PRO2 -> COM4 ;
PRO2 -> PRO8 ;
PRO2 -> PRO3 ;
PRO3 -> label ;
PRO3 -> PRO8 ;
PRO3 -> COM8 ;
PRO3 -> LAB56 ;
PRO3 -> LAB25 ;
PRO4 -> label ;
PRO4 -> PRO7 ;
PRO4 -> C

In [10]:
auc_list = []
recall_list = []
acc_list = []
#测试更新后的性能
for i in range(10):
#     print("各个客户端的初始性能")
    acc,recall,auc_score = cross_validation(clients[i].model,client_data,clients[i].cid)
    auc_list.append(auc_score)
    recall_list.append(recall)
    acc_list.append(acc)
#     print(i)
#     print_dag(clients[i].model)
# print("更新auc",auc_list)
# print("更新recall",recall_list)
# print("更新acc",acc_list)

    
    

  0%|          | 0/1284 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/744 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

...

  0%|          | 0/317 [00:00<?, ?it/s]

  0%|          | 0/428 [00:00<?, ?it/s]

[[0.85046729]
 [0.82617587]
 [0.84139785]
 [0.78005115]
 [0.7942029 ]
 [0.81113801]
 [0.72485207]
 [0.76328052]
 [0.7318612 ]
 [0.82476636]]
[[0.72972973]
 [0.71895425]
 [0.77294686]
 [0.86725664]
 [0.76691729]
 [0.74107143]
 [0.73913043]
 [0.40967742]
 [0.31578947]
 [0.72268908]]
[[0.81453664]
 [0.79697712]
 [0.82036542]
 [0.80593048]
 [0.78911902]
 [0.78914037]
 [0.72706522]
 [0.65831184]
 [0.61284969]
 [0.79338337]]


  0%|          | 0/1284 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [21]:
auc_list = np.atleast_2d(auc_list)

In [22]:
auc_result = pd.DataFrame(auc_list).round(4)   
# recall_result = pd.DataFrame(recall_list).round(4)
# acc_result = pd.DataFrame(acc_list).round(4)

ValueError: Must pass 2-d input. shape=(10, 10, 1)

In [ ]:
auc_result.to_csv("C:/Users/Administrator/Desktop/code/project_2/结果/auc.csv")
recall_result.to_csv("C:/Users/Administrator/Desktop/code/project_2/结果/recall.csv")
acc_result.to_csv("C:/Users/Administrator/Desktop/code/project_2/结果/acc.csv")

In [33]:
for i  in fuseDAG.edges():
    print(i[0],"->",i[1],";")

LAB19 -> label ;
COM1 -> label ;
COM2 -> label ;
COM3 -> label ;
COM4 -> label ;
COM5 -> label ;
COM6 -> label ;
COM7 -> label ;
COM8 -> label ;
COM9 -> label ;


In [ ]:
print_dag(fuseDAG)

In [ ]:
for i in range(client_num):
    #将客户端最好的模型边信息转换为矩阵形式
    edges_matrix+=to_matrix(clients[i].get_model().edges(),feature)


In [ ]:
for i in range(10):
    print("第",i)
    print_dag(clients[i].model)

In [33]:
for i,j in zip(hos_id,client_data):
    path = "C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data"+str(i)+".csv"
    print(path)
    j.to_csv(path)

C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data420.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data142.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data122.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data435.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data390.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data227.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data195.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data243.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data403.csv
C:/Users/Administrator/Desktop/代码/实验/fd/fd_data/data141.csv
